<em><sub>This page is available as an executable or viewable <strong>Jupyter Notebook</strong></sub></em>
<br/><br/>
<a href="https://mybinder.org/v2/gh/avan1235/KotlinDL/notebooks?filepath=docs%2Fpython_train_export_on_cifar_10.ipynb"
   target="_parent">
   <img align="left"
        src="https://mybinder.org/badge_logo.svg"
        height="20">
</a>
<a href="https://nbviewer.jupyter.org/github/avan1235/KotlinDL/blob/notebooks/docs/python_train_export_on_cifar_10.ipynb"
   target="_parent">
   <img align="right"
        src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg"
        height="20">
</a>
<br/><br/>

# Exporting Python Keras model

In this scenario we assume that we already have some code with neural netowork definition in Python that includes its training and data extraction. There are a few common scenarios when we would like to use this code indirecly and exporting the model seems to be the easiest approach.

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

As an example, we use the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) to train our neural network in Python which doesn't need a lot of preprocessing work as it is already prepared by Keras.

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

170500096/170498071 [==============================] - 2s 0us/step


We define our neural network as a common convolutional architecture with maxpooling layers for decreasing the image size. We end up with 10 outputs from the network as there are 10 labels to be detected in the dataset. 

In [3]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)), 
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 73s 46ms/step - loss: 1.5299 - accuracy: 0.4419 - val_loss: 1.2939 - val_accuracy: 0.5346
Epoch 2/10
1563/1563 [==============================] - 71s 46ms/step - loss: 1.1845 - accuracy: 0.5820 - val_loss: 1.0751 - val_accuracy: 0.6182
Epoch 3/10
1563/1563 [==============================] - 71s 45ms/step - loss: 1.0344 - accuracy: 0.6357 - val_loss: 1.0386 - val_accuracy: 0.6337
Epoch 4/10
1563/1563 [==============================] - 70s 45ms/step - loss: 0.9355 - accuracy: 0.6711 - val_loss: 0.9571 - val_accuracy: 0.6666
Epoch 5/10
1563/1563 [==============================] - 69s 44ms/step - loss: 0.8643 - accuracy: 0.6963 - val_loss: 0.9385 - val_accuracy: 0.6695
Epoch 6/10
1563/1563 [==============================] - 70s 45ms/step - loss: 0.8087 - accuracy: 0.7180 - val_loss: 0.8962 - val_accuracy: 0.6837
Epoch 7/10
1563/1563 [==============================] - 71s 45ms/step - loss: 0.7556 - accuracy: 0.7336 - val_loss: 0.8881 -

In [6]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 4s - loss: 0.8754 - accuracy: 0.7071
0.707099974155426


After training process done with `fit` method we saw that the evaluated model achieves over 70% of accuracy on the unseen test data.

Now we can save this model structure to `.json` file as well the weights form the model that we obtained during training process to standard data file format named `Hierarchical Data Format` or just `HDF5`.

In [7]:
model.save('keras-cifar-10/weights', save_format='h5')

model_json = model.to_json()
with open("keras-cifar-10/model.json", "w") as json_file:
    json_file.write(model_json)